### Import modules

In [1]:
import tensorflow as tf

# from https://medium.com/ibm-data-ai/memory-hygiene-with-tensorflow-during-model-training-and-deployment-for-inference-45cf49a15688
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(str(gpu))
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])


import tensorflow.keras as keras
from keras import layers
import numpy as np
from dataset import *

2023-01-20 19:49:18.319149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 19:49:18.511681: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-20 19:49:19.065081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-20 19:49:19.065152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


#### GPU config

### Create dataset from test audio file


In [2]:
xf, X, spec_params = create_dataset('audio/training_audio.wav', 2, 0.5)
X = np.abs(X)
print(X.shape)

STRIDE IS 0.007797696856520386
(116, 256, 256)


## Define model

In [3]:
POOLS = 5
inputs = keras.Input((256, 256, 1))

# Encoder - Conv2D gradually down to LSTM
# Conv1D didn't work as expected - instead, using conv2d but width of sliding window is 1

filter_expand = inputs
for i in range(POOLS):
    batch_norm = layers.BatchNormalization()(filter_expand)
    conv = layers.Conv2D(2**i, (1, 3), (1, 1), padding='same', activation='relu')(batch_norm)
    # add
    pool = layers.MaxPool2D((1, 2), (1, 2))(conv)   # (add)
    filter_expand = layers.Conv2D(2**(i+1), (1, 3), (1, 1), padding='same', activation='relu')(pool)

lstm_input = tf.reshape(filter_expand, (-1, 256, 256))

lstm = layers.LSTM(64, return_sequences=True)(lstm_input)
dense = layers.Dense(2**8)(lstm)
dense_reshaped = tf.reshape(dense, (-1, 256, 256//2**POOLS, 2**POOLS))

conv = dense_reshaped
for i in range(POOLS):
    batch_norm = layers.BatchNormalization()(conv)
    filter_reduce = layers.Conv2D(2**(POOLS - i - 1), (1, 3), (1,1), padding='same', activation='relu')(batch_norm)
    depool = layers.UpSampling2D((1, 2))(filter_reduce)
    conv = layers.Conv2D(2**(POOLS - i - 1), (1, 3), (1, 1), padding='same', activation='relu')(depool)

outputs = layers.Activation('sigmoid')(conv)
model = keras.Model(inputs=inputs, outputs=outputs, name="conv1d-lstm")

for layer in model.layers[-10:]:
    print(layer.output_shape)


2023-01-20 19:50:09.089502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 19:50:09.091229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-20 19:50:09.091838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-20 19:50:09.092307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

(None, 256, 64, 4)
(None, 256, 64, 4)
(None, 256, 64, 2)
(None, 256, 128, 2)
(None, 256, 128, 2)
(None, 256, 128, 2)
(None, 256, 128, 1)
(None, 256, 256, 1)
(None, 256, 256, 1)
(None, 256, 256, 1)


In [4]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "conv1d-lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 1)      4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 1)       4         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 128, 1)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 128, 2)       8         
                                                                 
 batch_normalization_1 (Batc  (None, 256, 128, 2)      

In [6]:
X_fit = X
X_fit = (X_fit - np.min(X_fit))/(np.max(X_fit) - np.min(X_fit))
X_fit = tf.expand_dims(X_fit, -1)
print("==================")
print(X_fit.shape)
print("==================")


(116, 256, 256, 1)


In [ ]:
model.load_weights("lstm_autoencoder")

In [7]:
model.fit(X_fit, X_fit,
        epochs=200,
        shuffle=True
)

Epoch 1/200


2023-01-20 19:51:38.791399: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
2023-01-20 19:51:39.976349: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-20 19:51:40.656793: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-20 19:51:40.784606: W tensorflow/core/framework/op_kernel.cc:1768] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError: Graph execution error:

Detected at node 'mean_squared_error/SquaredDifference' defined at (most recent call last):
    File "/home/nash/miniconda3/envs/tf/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/nash/miniconda3/envs/tf/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/nash/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/nash/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/nash/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/nash/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/nash/miniconda3/envs/tf/lib/python3.8/asyn

### COMPARE SPECTROGRAMS IN IMAGES


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x_example = X[0]

plt.imsave("INPUT_EXAMPLE.png", x_example)
x_example = np.reshape(x_example, (1, x_example.shape[0], x_example.shape[1], 1))
print(x_example.shape)
prediction = model.predict(x_example)
plt.imsave("OUTPUT_EXAMPLE.png", prediction[0, :, :, 0])

In [ ]:
model.save("cnn_autoencoder")

### Save to audio file

In [ ]:
import importlib
import postprocessing

In [ ]:
importlib.reload(postprocessing)
out_samp, out_win, out_stride = spec_params
audio = postprocessing.reverse_spectrogram(prediction[0, :, :, 0], out_samp, out_win, out_stride)
audio = np.reshape(audio, (-1, 1))
f_out = open_write("test_output_cnn.wav", 1, 2, 44100)
write(f_out, audio)